#Prepare

##Load Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_excel('/content/drive/MyDrive/DS310.M11/Data/dataset.xlsx')

In [ ]:
X=df.loc[:,'Comment']

In [ ]:
y=df.loc[:,'Label']
y.unique()

array([1, 0, 2])

##Preprocessing

In [ ]:
import unicodedata
import re

In [ ]:
pip install python-rdrsegmenter

     |████████████████████████████████| 259 kB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 48.3 MB/s 


In [ ]:
from python_rdrsegmenter import load_segmenter
rdrsegmenter= load_segmenter()

In [ ]:
def convert_unicode(cmt):
    cmt = unicodedata.normalize('NFC', cmt)
    return cmt

In [ ]:
def comment_preprocess(cmt):
    # chuẩn hóa unicode
    cmt = convert_unicode(cmt)
    # chuẩn hóa cách gõ dấu tiếng Việt (òa -> oà, úy -> uý,...) và tách từ
    cmt = rdrsegmenter.tokenize(cmt)
    #cmt = ' '.join(' '.join(word for word in sentence) for sentence in cmt)
    # đưa về lower
    cmt = cmt.lower()
    # xóa các ký tự không cần thiết
    cmt = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',cmt)
    # xóa khoảng trắng thừa
    cmt = re.sub(r'\s+', ' ', cmt).strip()
    return cmt

In [ ]:
for i in range(0,len(X)):
  X[i]=comment_preprocess(X[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
X

0       hình_ảnh không liên_quan và mang tính_chất nhậ...
1       đồ xịn jskakakeowowjdndndjdjdskkwlsnxnxjxksnsb...
2       hsbxbxnkwjdbx nxbxnwkkxnx nsjzhxbwjwijxnd bxbw...
3       vải mỏng size l mà bé tí nhắn_tin shop shop tr...
4       vải hơi mỏng nhưng so với giá tiền thì thấy cũ...
                              ...                        
9433    giấy mỏng lắm trông lm thì dễ nhưng khi làm th...
9434                chất_lượng sản_phẩm kém giấy quá mỏng
9435                            giấy mỏng dínhh thất_vọng
9436                        giấy mỏng đụng nhẹ là rách tệ
9437        chả dán dc gì keo ít dán xấu còn hơn chưa dán
Name: Comment, Length: 9438, dtype: object

##Teencode

In [ ]:
import pandas as pd
teencode_df = pd.read_csv('/content/drive/MyDrive/PTTQ/readme/teencode.txt',names=['teencode','map'],sep='\t',)

In [ ]:
teencode_list = teencode_df['teencode'].to_list()
map_list = teencode_df['map'].to_list()

In [ ]:
def searchTeencode(word):
  try:
    global teencode_count
    index = teencode_list.index(word)
    map_word = map_list[index]
    teencode_count += 1
    return map_word
  except:
    pass

#Stopwords

In [ ]:
STOPWORDS = '/content/drive/MyDrive/PTTQ/readme/vietnamese-stopwords-dash.txt'

In [ ]:
# features extraction
with open(STOPWORDS, "r") as ins:
    stopword = []
    for line in ins:
        stopword.append(line.strip('\n'))

In [ ]:
def remove_stopwords(line):
    global stopword_count
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
        if word in stopword:
            stopword_count += 1
    return ' '.join(words)

In [ ]:
!pip install underthesea

from underthesea import word_tokenize

     |████████████████████████████████| 7.5 MB 4.3 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 3.4 MB 42.8 MB/s 
     |████████████████████████████████| 743 kB 44.8 MB/s 
     |████████████████████████████████| 235 kB 54.3 MB/s 
     |████████████████████████████████| 895 kB 53.8 MB/s 
     |████████████████████████████████| 3.3 MB 42.9 MB/s 
     |████████████████████████████████| 61 kB 376 kB/s 
     |████████████████████████████████| 596 kB 43.3 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=8ad46daf5b67e15a46a98aab16ea5804a6f75c13a39bd074d3645bb1c4f1cfa5
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Combine Teencode and Stopwords

In [ ]:
stopword_count = 0
teencode_count =0
def StopWords_Teencode(sentence):
  lenn = 0
  sentence = str(sentence)
  #Tokenize
  List_tokens = word_tokenize(sentence,format='text')
  List_tokens = word_tokenize(List_tokens)

  #Teencode
  for tokens_idx, text_tokens in enumerate(List_tokens):
    deteencoded = searchTeencode(text_tokens)
    if (deteencoded != None):
        List_tokens[tokens_idx] = deteencoded

  deteencode_sentence = (" ").join(List_tokens)
  
  #Stopwords
  tokens_without_sw = remove_stopwords(deteencode_sentence)

  return tokens_without_sw

In [ ]:
for i in range(0,len(X)-1):
  X[i]=StopWords_Teencode(X[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
X

0       hình_ảnh tính_chất xu chất vải tạm ổn giá giá ...
1       đồ xịn jskakakeowowjdndndjdjdskkwlsnxnxjxksnsb...
2       hsbxbxnkwjdbx nxbxnwkkxnx nsjzhxbwjwijxnd bxbw...
3       vải mỏng size l bé_tí nhắn_tin_shop_shop trả_l...
4       vải hơi mỏng giá tiền oke shop tặng kt dịch gi...
                              ...                        
9433    giấy mỏng lắm trông toàn nhăn_nheo trông thẩm_...
9434                    chất_lượng sản_phẩm kém giấy mỏng
9435                            giấy mỏng dínhh thất_vọng
9436                           giấy mỏng đụng nhẹ rách tệ
9437        chả dán dc gì keo ít dán xấu còn hơn chưa dán
Name: Comment, Length: 9438, dtype: object

##Chia Train, Test, Dev

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#Machine Learning

##Naive Bayes

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(analyzer='word', ngram_range=(2,2))
tf_idf.fit(X_train)

X_train_encoded = tf_idf.transform(X_train)
X_dev_encoded = tf_idf.transform(X_dev)
X_test_encoded = tf_idf.transform(X_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
model.fit(X_train_encoded, y_train)

MultinomialNB()

In [ ]:
y_pred_dev = model.predict(X_dev_encoded)
y_pred_test = model.predict(X_test_encoded)

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(y_pred_dev, y_dev, average='macro')*100)
print(f1_score(y_pred_test, y_test, average='macro')*100)

36.1450089753483
35.11787677173304


In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred_dev, y_dev)*100)
print(accuracy_score(y_pred_test, y_test)*100)

65.23178807947019
65.2542372881356


##SVM

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(analyzer='word', ngram_range=(2,2))
tf_idf.fit(X_train)

X_train_encoded = tf_idf.transform(X_train)
X_dev_encoded = tf_idf.transform(X_dev)
X_test_encoded = tf_idf.transform(X_test)

In [ ]:
from sklearn import svm

model = svm.SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
model.fit(X_train_encoded, y_train)

SVC()

In [ ]:
y_pred_dev = model.predict(X_dev_encoded)
y_pred_test = model.predict(X_test_encoded)

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(y_pred_dev, y_dev, average='macro')*100)
print(f1_score(y_pred_test, y_test, average='macro')*100)

44.68798610358398
41.436341436341436


In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred_dev, y_dev)*100)
print(accuracy_score(y_pred_test, y_test)*100)

67.1523178807947
66.6843220338983


##Logistic Regression

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(analyzer='word', ngram_range=(2,2))
tf_idf.fit(X_train)

X_train_encoded = tf_idf.transform(X_train)
X_dev_encoded = tf_idf.transform(X_dev)
X_test_encoded = tf_idf.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression as LR

model = LR(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
model.fit(X_train_encoded, y_train)

LogisticRegression()

In [ ]:
y_pred_dev = model.predict(X_dev_encoded)
y_pred_test = model.predict(X_test_encoded)

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(y_pred_dev, y_dev, average='macro')*100)
print(f1_score(y_pred_test, y_test, average='macro')*100)

48.5739112696629
47.66014739965595


In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred_dev, y_dev)*100)
print(accuracy_score(y_pred_test, y_test)*100)

67.74834437086092
67.79661016949152


#DeepLearning

In [ ]:
pip install pyvi

     |████████████████████████████████| 8.5 MB 4.1 MB/s 


In [ ]:
# Xây dựng tập từ vựng
from pyvi import ViTokenizer

V = []

for t in X_train:
    tokenized_sentence = ViTokenizer.tokenize(t)
    V = V + tokenized_sentence.split()

V = list(set(V))

In [ ]:
# Building dictionary 
word_to_index = {w : (i+2) for i, w in enumerate(V)}

word_to_index['UNK'] = 1
word_to_index['PAD'] = 0

# Build index_to_word
index_to_word = {i: w for w, i in word_to_index.items()}

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pyvi import ViTokenizer

max_len = 100

def encoding(X):
    sentences = []
    
    for t in X:
        # tach tu 
        tokenized_sentence = ViTokenizer.tokenize(t)
        sentences.append(tokenized_sentence)
    
    X = []
    for s in sentences:
        sent = []
        for w in s.split():
            try:
                sent.append(word_to_index[w])
            except:
                sent.append(word_to_index["UNK"])
        X.append(sent)
    
    # Padding du lieu theo do dai cau (maxlen)
    X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index["PAD"])

    return X

In [ ]:
X_train_encoded = encoding(X_train)
X_dev_encoded = encoding(X_dev)
X_test_encoded = encoding(X_test)

##Neural Network

In [ ]:
from keras.layers import Dense, Embedding, Flatten
from keras.models import Model, Input
from keras.initializers import Constant

num_words = len(word_to_index)
input = Input(shape = (max_len, ))
emb = Embedding(input_dim=num_words+1,
                    output_dim=300,
                    input_length=max_len)(input)
flat = Flatten()(emb)
output = Dense(3, activation="sigmoid")(flat)

model = Model(input, output)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 300)          2122500   
                                                                 
 flatten (Flatten)           (None, 30000)             0         
                                                                 
 dense (Dense)               (None, 3)                 90003     
                                                                 
Total params: 2,212,503
Trainable params: 2,212,503
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.utils import to_categorical

model.fit(X_train_encoded, to_categorical(y_train, num_classes=3), 
          validation_data=(X_dev_encoded, to_categorical(y_dev, num_classes=3)), 
          batch_size=128, epochs=10)

Epoch 1/10
48/48 [==============================] - 4s 28ms/step - loss: 0.8825 - categorical_accuracy: 0.6379 - val_loss: 0.8197 - val_categorical_accuracy: 0.6596
Epoch 2/10
48/48 [==============================] - 1s 15ms/step - loss: 0.6568 - categorical_accuracy: 0.7180 - val_loss: 0.6385 - val_categorical_accuracy: 0.7252
Epoch 3/10
48/48 [==============================] - 1s 17ms/step - loss: 0.4750 - categorical_accuracy: 0.8063 - val_loss: 0.5962 - val_categorical_accuracy: 0.7503
Epoch 4/10
48/48 [==============================] - 1s 16ms/step - loss: 0.3587 - categorical_accuracy: 0.8642 - val_loss: 0.5939 - val_categorical_accuracy: 0.7404
Epoch 5/10
48/48 [==============================] - 1s 16ms/step - loss: 0.2736 - categorical_accuracy: 0.9053 - val_loss: 0.6128 - val_categorical_accuracy: 0.7364
Epoch 6/10
48/48 [==============================] - 1s 16ms/step - loss: 0.2148 - categorical_accuracy: 0.9311 - val_loss: 0.6230 - val_categorical_accuracy: 0.7311
Epoch 7/10

In [ ]:
import numpy as np

y_pred_test = model.predict(X_test_encoded)
y_pred_test = np.argmax(y_pred_test, axis=-1)

In [ ]:
new_df={'Comment':X_test,'Rate':y_test,'Predicted':y_pred_test}

In [ ]:
newdf=pd.DataFrame(new_df)

In [ ]:
newdf.to_excel('11.xlsx')

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(y_pred_test, y_test, average='macro')*100)

66.58228462548519


In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred_test, y_test)*100)

72.72245762711864


##Bi-GRU

In [ ]:
from keras.layers import Dense, Embedding, Flatten, GRU
from keras.models import Model, Input
from keras.initializers import Constant
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

num_words = len(word_to_index)
input = Input(shape = (max_len, ))
emb = Embedding(input_dim=num_words+1,
                    output_dim=300,
                    input_length=max_len)(input)
model = SpatialDropout1D(0.1)(emb)
model = Bidirectional(GRU(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
flat = Flatten()(model)

output = Dense(3, activation="softmax")(flat)
model = Model(input, output)

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 300)          2122500   
                                                                 
 spatial_dropout1d (SpatialD  (None, 100, 300)         0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 100, 200)         241200    
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 20000)             0         
                                                                 
 dense (Dense)               (None, 3)                 60003 

In [ ]:
from tensorflow.keras.utils import to_categorical

model.fit(X_train_encoded, to_categorical(y_train, num_classes=3), 
          validation_data=(X_dev_encoded, to_categorical(y_dev, num_classes=3)), 
          batch_size=128, epochs=10)

Epoch 1/10
48/48 [==============================] - 102s 2s/step - loss: 0.8411 - categorical_accuracy: 0.6512 - val_loss: 0.6529 - val_categorical_accuracy: 0.7146
Epoch 2/10
48/48 [==============================] - 46s 960ms/step - loss: 0.5504 - categorical_accuracy: 0.7707 - val_loss: 0.5909 - val_categorical_accuracy: 0.7358
Epoch 3/10
48/48 [==============================] - 46s 950ms/step - loss: 0.3974 - categorical_accuracy: 0.8328 - val_loss: 0.6179 - val_categorical_accuracy: 0.7417
Epoch 4/10
48/48 [==============================] - 47s 971ms/step - loss: 0.3060 - categorical_accuracy: 0.8772 - val_loss: 0.6906 - val_categorical_accuracy: 0.7272
Epoch 5/10
48/48 [==============================] - 46s 959ms/step - loss: 0.2378 - categorical_accuracy: 0.9035 - val_loss: 0.8181 - val_categorical_accuracy: 0.7371
Epoch 6/10
48/48 [==============================] - 45s 944ms/step - loss: 0.1980 - categorical_accuracy: 0.9180 - val_loss: 0.9165 - val_categorical_accuracy: 0.7252


In [ ]:
import numpy as np

y_pred_test = model.predict(X_test_encoded)
y_pred_test = np.argmax(y_pred_test, axis=-1)

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(y_pred_test, y_test, average='macro')*100)

62.001905798473864


In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred_test, y_test)*100)

69.01483050847457


##Bi-LSTM

In [ ]:
from keras.layers import Dense, Embedding, Flatten, LSTM
from keras.models import Model, Input
from keras.initializers import Constant
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

num_words = len(word_to_index)
input = Input(shape = (max_len, ))
emb = Embedding(input_dim=num_words+1,
                    output_dim=300,
                    input_length=max_len)(input)
model = SpatialDropout1D(0.1)(emb)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
flat = Flatten()(model)

output = Dense(3, activation="softmax")(flat)
model = Model(input, output)

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.summary()

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 100)]             0         
                                                                 
 embedding_20 (Embedding)    (None, 100, 300)          2122500   
                                                                 
 spatial_dropout1d_11 (Spati  (None, 100, 300)         0         
 alDropout1D)                                                    
                                                                 
 bidirectional_16 (Bidirecti  (None, 100, 200)         320800    
 onal)                                                           
                                                                 
 flatten_14 (Flatten)        (None, 20000)             0         
                                                                 
 dense_20 (Dense)            (None, 3)                 600

In [ ]:
from tensorflow.keras.utils import to_categorical

model.fit(X_train_encoded, to_categorical(y_train, num_classes=3), 
          validation_data=(X_dev_encoded, to_categorical(y_dev, num_classes=3)), 
          batch_size=128, epochs=10)

Epoch 1/10
48/48 [==============================] - 73s 1s/step - loss: 0.8569 - categorical_accuracy: 0.6429 - val_loss: 0.6749 - val_categorical_accuracy: 0.7000
Epoch 2/10
48/48 [==============================] - 62s 1s/step - loss: 0.5575 - categorical_accuracy: 0.7624 - val_loss: 0.6111 - val_categorical_accuracy: 0.7325
Epoch 3/10
48/48 [==============================] - 62s 1s/step - loss: 0.4084 - categorical_accuracy: 0.8278 - val_loss: 0.6234 - val_categorical_accuracy: 0.7371
Epoch 4/10
48/48 [==============================] - 61s 1s/step - loss: 0.3121 - categorical_accuracy: 0.8717 - val_loss: 0.6958 - val_categorical_accuracy: 0.7298
Epoch 5/10
48/48 [==============================] - 62s 1s/step - loss: 0.2455 - categorical_accuracy: 0.9010 - val_loss: 0.8689 - val_categorical_accuracy: 0.6609
Epoch 6/10
48/48 [==============================] - 61s 1s/step - loss: 0.2179 - categorical_accuracy: 0.9147 - val_loss: 0.8731 - val_categorical_accuracy: 0.7325
Epoch 7/10
48/48

In [ ]:
import numpy as np

y_pred_test = model.predict(X_test_encoded)
y_pred_test = np.argmax(y_pred_test, axis=-1)

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(y_pred_test, y_test, average='macro')*100)

62.85312183258166


In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred_test, y_test)*100)

70.23305084745762
